# UDP creation for model inference using openEO

This notebook demonstrates how to use create a sharable UDP.
First we establish a connection and load in he required libraries

In [ ]:
import json
import openeo
from openeo.api.process import Parameter
from utils import  preprocess_sentinel2_data
from openeo.rest.udp import build_process_dict


#establish connection
#conn = openeo.connect("https://openeo.cloud/").authenticate_oidc()
conn = openeo.connect("https://openeo.dataspace.copernicus.eu/").authenticate_oidc()

Create the process graph

Changeable parameters include; the spatial extend, the temporal extend and the model url.

In [ ]:
spatial_extent_param = Parameter(
    name="spatial_extend",
    description="The bounding box to load.",
    schema={
    "type": "object",
    "properties": {
        "west": {"type": "number"},
        "south": {"type": "number"},
        "east": {"type": "number"},
        "north": {"type": "number"},
        "crs": {"type": "string"}
    }
},
)

temporal_extent = Parameter(
    name="temporal_extent",
    description="The date range to calculate the temporal mean for.",
    schema={"type": "array", "subtype": "temporal-interval"},
    default =["2023-05-01", "2023-09-30"])

inference_url_param = Parameter.string(
    name="inference_url",
    default="https://artifactory.vgt.vito.be/artifactory/auxdata-public/photovoltaic/rf_1_median_depth_15.onnx",
    description="url to the inference network, must be on VITO artifactory",
)

Create the processing graph

In [ ]:
from utils import preprocess_sentinel2_data, postprocess_inference_data

s2_cube = preprocess_sentinel2_data(conn,  spatial_extent_param, temporal_extent)

# Supply the model as a URL. The model is stored in artifactory
udf = openeo.UDF.from_file(
    "udf_rf_onnx.py", 
    context={
        "model_url": inference_url_param
    }
)

# Reduce the bands dimension using the inference udf
prediction = s2_cube.reduce_bands(reducer=udf)
prediction_filtered = postprocess_inference_data(prediction, 3) 

Store the process graph on the back-end and locally

In [ ]:
#save the graph on the back-end
conn.save_user_defined_process(
    user_defined_process_id="pv_inference",
    process_graph=prediction,
    parameters=[spatial_extent_param, temporal_extent, inference_url_param]
)

#save it locally as a JSON
spec = build_process_dict(
    process_id="pv_inference",
    process_graph=prediction,
    parameters=[spatial_extent_param, temporal_extent, inference_url_param]

)
with open("pv_inference.json", "w") as f:
    json.dump(spec, f, indent=2)

Add the onnx dependencies to the job options. You can reuse this existing dependencies archive.
Excecute the batch job.

In [ ]:

dependencies_url = "https://artifactory.vgt.vito.be/artifactory/auxdata-public/openeo/onnx_dependencies_1.16.3.zip"
job_options = {
    "udf-dependency-archives": [
        f"{dependencies_url}#onnx_deps" #onnx_deps is not a comment, but part of the string. Do not remove or add spaces.
    ],
}

inference_result = conn.datacube_from_process(
    process_id="pv_inference",
    spatial_extend={"west": 12.17,
                    "east": 12.18,
                    "south": 51.46,
                    "north": 51.47},
    temporal_extend = ["2023-05-01", "2023-09-30"],
    inference_url = "https://artifactory.vgt.vito.be/artifactory/auxdata-public/photovoltaic/rf_1_median_depth_15.onnx")

inference_result.execute_batch(
    "./photovoltaic_prediction.nc",
    job_options=job_options,
    title="photovoltaic_prediction"
)